In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/queries/resolved_queries.csv
/kaggle/input/queries/new_queries.csv


In [11]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
##PART1

In [4]:
# Load the CSV files
resolved_queries_path = '/kaggle/input/queries/resolved_queries.csv'
new_queries_path = '/kaggle/input/queries/new_queries.csv'

resolved_queries = pd.read_csv(resolved_queries_path)
new_queries = pd.read_csv(new_queries_path)

resolved_queries.head()

,Query_ID,Pre_Resolved_Query
0,1,Unable to connect to the internet
1,2,Payment failed during checkout
2,3,App crashes when opening settings
3,4,Forgot password and unable to reset
4,5,Unable to upload files to the server


In [5]:
new_queries.head()

,Variation_Query,Matches_With_Query_ID
0,Unabel to conect to the internet,1
1,Can’t connect to internet,1
2,Intenet not working,1
3,Payment failed while chekout,2
4,Payment did not go through during chckout,2


In [7]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove any punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text


resolved_queries['Processed_Query'] = resolved_queries['Pre_Resolved_Query'].apply(preprocess_text)
new_queries['Processed_Variation_Query'] = new_queries['Variation_Query'].apply(preprocess_text)


resolved_queries[['Pre_Resolved_Query', 'Processed_Query']].head(), new_queries[['Variation_Query', 'Processed_Variation_Query']].head()

(                     Pre_Resolved_Query                       Processed_Query
 0     Unable to connect to the internet     unable to connect to the internet
 1        Payment failed during checkout        payment failed during checkout
 2     App crashes when opening settings     app crashes when opening settings
 3   Forgot password and unable to reset   forgot password and unable to reset
 4  Unable to upload files to the server  unable to upload files to the server,
                              Variation_Query  \
 0           Unabel to conect to the internet   
 1                  Can’t connect to internet   
 2                        Intenet not working   
 3               Payment failed while chekout   
 4  Payment did not go through during chckout   
 
                    Processed_Variation_Query  
 0           unabel to conect to the internet  
 1                   cant connect to internet  
 2                        intenet not working  
 3               payment failed while

In [9]:
def fuzzy_match(variation_query, resolved_queries, method='ratio'):
    if method == 'ratio':
        # Use simple Levenshtein distance-based ratio
        scores = resolved_queries['Processed_Query'].apply(lambda x: fuzz.ratio(variation_query, x))
    elif method == 'partial_ratio':
        # Use partial ratio
        scores = resolved_queries['Processed_Query'].apply(lambda x: fuzz.partial_ratio(variation_query, x))
    elif method == 'token_sort_ratio':
        # Use token sort ratio
        scores = resolved_queries['Processed_Query'].apply(lambda x: fuzz.token_sort_ratio(variation_query, x))
    elif method == 'token_set_ratio':
        # Use token set ratio
        scores = resolved_queries['Processed_Query'].apply(lambda x: fuzz.token_set_ratio(variation_query, x))
    
    # Find the best match
    best_match_idx = scores.idxmax()
    best_match_score = scores.max()
    best_match_query_id = resolved_queries.iloc[best_match_idx]['Query_ID']
    
    return best_match_query_id, best_match_score

In [10]:
# Test with different fuzzy matching methods
methods = ['ratio', 'partial_ratio', 'token_sort_ratio', 'token_set_ratio']
results = {}

for method in methods:
    results[method] = new_queries['Processed_Variation_Query'].apply(lambda x: fuzzy_match(x, resolved_queries, method))

# Extracting results for comparison
fuzzy_results = pd.DataFrame({
    'Variation_Query': new_queries['Variation_Query'],
    'Matched_Query_ID_Ratio': [result[0] for result in results['ratio']],
    'Match_Score_Ratio': [result[1] for result in results['ratio']],
    'Matched_Query_ID_Partial_Ratio': [result[0] for result in results['partial_ratio']],
    'Match_Score_Partial_Ratio': [result[1] for result in results['partial_ratio']],
    'Matched_Query_ID_Token_Sort_Ratio': [result[0] for result in results['token_sort_ratio']],
    'Match_Score_Token_Sort_Ratio': [result[1] for result in results['token_sort_ratio']],
    'Matched_Query_ID_Token_Set_Ratio': [result[0] for result in results['token_set_ratio']],
    'Match_Score_Token_Set_Ratio': [result[1] for result in results['token_set_ratio']],
})

fuzzy_results.head()

,Variation_Query,Matched_Query_ID_Ratio,Match_Score_Ratio,Matched_Query_ID_Partial_Ratio,Match_Score_Partial_Ratio,Matched_Query_ID_Token_Sort_Ratio,Match_Score_Token_Sort_Ratio,Matched_Query_ID_Token_Set_Ratio,Match_Score_Token_Set_Ratio
0,Unabel to conect to the internet,1,95,1,94,1,95,1,95
1,Can’t connect to internet,1,77,1,83,1,67,1,88
2,Intenet not working,2,33,1,52,1,35,1,37
3,Payment failed while chekout,2,83,2,79,2,76,2,83
4,Payment did not go through during chckout,2,68,2,61,2,65,2,68


In [12]:
# Vectorize the queries using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_resolved = vectorizer.fit_transform(resolved_queries['Processed_Query'])
tfidf_new = vectorizer.transform(new_queries['Processed_Variation_Query'])

# Compute cosine similarity between each new query and all resolved queries
cosine_similarities = cosine_similarity(tfidf_new, tfidf_resolved)

# Find the best match for each new query based on cosine similarity
best_match_indices = cosine_similarities.argmax(axis=1)
best_match_scores = cosine_similarities.max(axis=1)

# Map the indices back to the resolved Query_IDs
matched_query_ids = resolved_queries.iloc[best_match_indices]['Query_ID'].values

# Combine results with new queries
tfidf_results = pd.DataFrame({
    'Variation_Query': new_queries['Variation_Query'],
    'Matched_Query_ID_TFIDF': matched_query_ids,
    'Match_Score_TFIDF': best_match_scores
})

tfidf_results.head()

,Variation_Query,Matched_Query_ID_TFIDF,Match_Score_TFIDF
0,Unabel to conect to the internet,1,0.839042
1,Can’t connect to internet,1,0.836936
2,Intenet not working,1,0.000000
3,Payment failed while chekout,2,0.707107
4,Payment did not go through during chckout,2,0.707107


In [13]:
##PART 2

In [14]:
name_variations = pd.read_csv('/kaggle/input/textsearch/name_variations.csv', header=None, names=['Name_Variation'])
base_names = pd.read_csv('/kaggle/input/textsearch/base_names.csv', header=None, names=['Base_Name'])

In [15]:
name_variations.head()

,Name_Variation
Variation,Matches_With_Base_Name
Thomas King,Thomas King
ThomasKing,Thomas King
Maria Garcia,Maria Garcia
MaryLewis,Mary Lewis


In [16]:
base_names.head()

,Base_Name
Base_Name_ID,Base_Name
1,John Smith
2,Jennifer Brown
3,Michael O'Connor
4,Maria Garcia


In [17]:
# Preprocess the names: lowercasing and stripping extra spaces
name_variations['Processed_Name_Variation'] = name_variations['Name_Variation'].str.lower().str.strip()
base_names['Processed_Base_Name'] = base_names['Base_Name'].str.lower().str.strip()

In [18]:

# Function to match name variations with base names using fuzzy matching
def match_names(variation_name, base_names):
    # Apply token set ratio fuzzy matching
    match_score = base_names['Processed_Base_Name'].apply(lambda x: fuzz.token_set_ratio(variation_name, x))
    
    # Find the index of the best match
    best_match_idx = match_score.idxmax()
    
    # Get the highest score and corresponding base name
    best_match_score = match_score.max()
    best_match_name = base_names.iloc[best_match_idx]['Base_Name']
    
    return best_match_name, best_match_score

In [19]:
# Ensure the indices are reset
base_names = base_names.reset_index(drop=True)
name_variations = name_variations.reset_index(drop=True)

# Apply the matching function to the name variations
name_variations['Matched_Base_Name'], name_variations['Match_Score'] = zip(
    *name_variations['Processed_Name_Variation'].apply(lambda x: match_names(x, base_names))
)

# Display the results
print("\nMatching Results:")
print(name_variations[['Name_Variation', 'Matched_Base_Name', 'Match_Score']].head())
# Ensure the indices are reset
base_names = base_names.reset_index(drop=True)
name_variations = name_variations.reset_index(drop=True)

# Apply the matching function to the name variations
name_variations['Matched_Base_Name'], name_variations['Match_Score'] = zip(
    *name_variations['Processed_Name_Variation'].apply(lambda x: match_names(x, base_names))
)

# Display the results
print("\nMatching Results:")
print(name_variations[['Name_Variation', 'Matched_Base_Name', 'Match_Score']])


Matching Results:
           Name_Variation Matched_Base_Name  Match_Score
0  Matches_With_Base_Name         Base_Name           58
1             Thomas King       Thomas King          100
2             Thomas King       Thomas King          100
3            Maria Garcia      Maria Garcia          100
4              Mary Lewis        Mary Lewis          100

Matching Results:
             Name_Variation Matched_Base_Name  Match_Score
0    Matches_With_Base_Name         Base_Name           58
1               Thomas King       Thomas King          100
2               Thomas King       Thomas King          100
3              Maria Garcia      Maria Garcia          100
4                Mary Lewis        Mary Lewis          100
..                      ...               ...          ...
96           Jennifer Brown    Jennifer Brown          100
97             Daniel Scott      Daniel Scott          100
98           David Martinez    David Martinez          100
99               Paul Allen   